In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
file_path = Path("lending_data.csv")
df = pd.read_csv(file_path)

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [4]:
# Separate the data into labels and features

# Separate the y variable, the labels
y = df["loan_status"]

# Separate the X variable, the features
X = df.drop(columns="loan_status")

In [5]:
# Review the y variable Series
y.head()

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [6]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Split the data into training and testing datasets by using `train_test_split`.

In [8]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(58152, 7)

In [9]:
print(X_train.shape)
print(X_test.shape)
print(X.shape)

(58152, 7)
(19384, 7)
(77536, 7)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [10]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)
classifier

# Fit the model using training data
classifier.fit(X_train, y_train)

# Addit - score the model
print(f"Training data score: {classifier.score(X_train, y_train)}")
print(f"Testing data score: {classifier.score(X_test, y_test)}")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Training data score: 0.9914878250103177
Testing data score: 0.9924164259182832


### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [11]:
# Make a prediction using the testing data
predictions = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)
results.tail(10)

,Prediction,Actual
19374,1,1
19375,0,0
19376,0,0
19377,0,0
19378,0,0
19379,0,0
19380,0,0
19381,0,0
19382,0,0
19383,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Generate a confusion matrix.

* Print the classification report.

In [12]:
# accuracy score
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.9924164259182832

In [13]:
# Generate a confusion matrix for the model
confusion_matrix(y_test, predictions)

array([[18679,    80],
       [   67,   558]])

In [14]:
# Print the classification report for the model
target_names = ["loan status 0 (healthy)", "loan status 1 (unhealthy)"]
print(classification_report(y_test, predictions, target_names=target_names))

                           precision    recall  f1-score   support

  loan status 0 (healthy)       1.00      1.00      1.00     18759
loan status 1 (unhealthy)       0.87      0.89      0.88       625

                 accuracy                           0.99     19384
                macro avg       0.94      0.94      0.94     19384
             weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The logistic regression model predicts with 100% precision, all loans that are predicted to be healthy and that are actually healthy. This is the precision. Importantly, the recall for these loans (healthy) is also 100% which indicates that of loans that are actually healthy, all of them are predicted to be so. 

For unhealthy loans (status 1), precision and recall are reduced. The precision of 0.87 indicates that for all loans predicted to be unhealthy, 87% of them will be correct (i.e. unhealthy), however this also means that a proportion of the loans detected as unhealthy will actually be false positives, and therefore although detected as 'unhealthy' those false positives will actually mean those loans are 'healthy'.

My interpretation of this result is that the bank wants to 'overcall' it. That is, they are prepared to identify a proportion of false positives in the unhealthy loan group (ie. loans predicted as unhealthy, that are actually healthy) as this means that the pick up unhealthy loans is higher. Of note is the higher prediction and recall for healthy loans that indicates if the loan is predicted as 'healthy' it is extremely unlikely that it is in fact 'unhealthy' (and wrongly reassuringly identified as healthy by the model), as this would be a dangerous situation for the bank to be in (ie. unrecognised high risk loans present).

---